In [9]:
import cv2 
import mediapipe as mp 
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 

In [10]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/ np.pi)

    if angle > 180.0:
        angle = 360-angle
        
    return angle

In [11]:
def distance(x, y):
    return ((y[0] - x[0]) ** 2 + (y[1] - x[1]) ** 2) ** (1/2)

In [18]:
#VIDEO FEED
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
# Increase the accurance by changing the confidence number 
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        image = cv2.flip(image, 1)

        image.flags.writeable = False 

        # Make detection (accessing our pose model)
        # store our results in the variable
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks (joints)
        try:
            landmarks = results.pose_landmarks.landmark

            #Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
            #             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            # elbow = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            l_foot = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

            
            # Calculate angle
            angle_knee = calculate_angle(l_hip, l_knee, l_ankle)
            angle_ankle = calculate_angle(l_knee, l_ankle, l_foot)
            
            #get distance
            ankle_to_hip = distance(l_foot, l_hip)

            #Visualize angle
            cv2.putText(image, str(angle_knee),
                        tuple(np.multiply(l_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            cv2.putText(image, str(angle_ankle),
                        tuple(np.multiply(l_ankle, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            cv2.putText(image, str(ankle_to_hip),
                        tuple(np.multiply(l_hip, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Concate Row (if i want to add face or hand)
            row = pose_row

            # Append class name
            row.insert(0, class_name)

            with open('coords.csv', mode='w', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(landmarks)

            
        except:
            pass


        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  #color of the joint dot
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  #color of the connection between joints
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

AttributeError: 'RepeatedCompositeFieldContainer' object has no attribute 'process'

# Capture Landmarks & Export to CSV

In [5]:
import csv 
import os 
import numpy as np 

In [8]:
num_coords = len(results.pose_landmarks.landmark)

In [10]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [13]:
#create csv export columns to csv

with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)


In [12]:
landmarks[-1]

'v33'

In [17]:
class_name = "Pedaling"

In [20]:
#VIDEO FEED
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
# Increase the accurance by changing the confidence number 
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        image = cv2.flip(image, 1)

        image.flags.writeable = False 

        # Make detection (accessing our pose model)
        # store our results in the variable
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks (joints)
        try:
            landmarks = results.pose_landmarks.landmark

            #Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            l_foot = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, 
                      landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

            
            # Calculate angle
            angle_knee = calculate_angle(l_hip, l_knee, l_ankle)
            angle_ankle = calculate_angle(l_knee, l_ankle, l_foot)
            
            #get distance
            ankle_to_hip = distance(l_foot, l_hip)

            #Visualize angle
            cv2.putText(image, str(angle_knee),
                        tuple(np.multiply(l_knee, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            cv2.putText(image, str(angle_ankle),
                        tuple(np.multiply(l_ankle, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )
            cv2.putText(image, str(ankle_to_hip),
                        tuple(np.multiply(l_hip, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                        )


        except:
            pass


        # Render detection
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                  #color of the joint dot
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  #color of the connection between joints
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                  )

        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [19]:
cap.release()
cv2.destroyAllWindows()

In [18]:
np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()

array([ 0.58431733,  0.15713522, -1.47611237,  0.9998256 ,  0.62091458,
        0.0745687 , -1.4617902 ,  0.99976391,  0.6434561 ,  0.07192742,
       -1.46224618,  0.99980646,  0.66251075,  0.06853723, -1.46277845,
        0.99981481,  0.54399627,  0.07069651, -1.47170973,  0.99979222,
        0.51394784,  0.06537333, -1.47213662,  0.99979657,  0.48949116,
        0.06058053, -1.47243202,  0.99976599,  0.68504208,  0.05983482,
       -1.21251428,  0.9998799 ,  0.45330071,  0.05129431, -1.2613188 ,
        0.99979544,  0.61372578,  0.22259833, -1.36707246,  0.99983573,
        0.53739178,  0.226604  , -1.38114142,  0.99982351,  0.77437204,
        0.26630959, -0.98522741,  0.99982101,  0.30563623,  0.24625193,
       -1.00685251,  0.99862134,  0.85768253,  0.70254081, -0.93271667,
        0.95019537,  0.24618308,  0.70908475, -0.9296006 ,  0.86028612,
        0.94638312,  1.1095506 , -1.10267675,  0.76448482,  0.15032813,
        1.03849661, -1.02636516,  0.47720304,  1.01249576,  1.20

In [15]:
pose = results.pose_landmarks.landmark